SUUMOから物件情報を取得する。

参考文献 : 
https://www.youtube.com/watch?v=l9KRgSUeNXk
https://www.youtube.com/watch?v=gixuk9qFpRg
https://www.youtube.com/watch?v=_H9bdFOZ4vs
https://www.youtube.com/watch?v=Lq4xHWnT_KE 
https://qiita.com/tomyu/items/a08d3180b7cbe63667c9
https://qiita.com/tommy24/items/52048b98bc1169b95c05

東京都大田区
https://suumo.jp/chintai/tokyo/sc_ota/mansion/?page=1

利用規約
https://suumo.jp/edit/kiyaku/

取得する情報
- 物件情報
  - 物件名
  - 住所
  - アクセス
  - 築年数
- 部屋情報
  - 物件の階数
  - 物件の賃料/管理費
  - 物件の敷金・礼金
  - 物件の間取り・面積

In [ ]:
#SUUMOから物件情報を取得する。
#今回は東京都大田区
#ライブラリーのインポート
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
from tqdm import tqdm
import pandas as pd


#変数urlにSUUMOホームページのURLを格納する
url = 'https://suumo.jp/chintai/tokyo/sc_ota/mansion/?page={}'
#'Requests'を使って1で設定したURLにアクセスする
res = requests.get(url.format(1))
#取得したHTMLを'BeautifulSoup'で解析する
soup_total_page = BeautifulSoup(res.text, 'html.parser')
#最後のPage数を取得する
last_page = int(soup_total_page.find('ol', class_='pagination-parts').find_all('li')[-1].text)

#変数d_listにからのリストを作成する
d_list = []

#1〜3をループする
for i in tqdm(range(1, last_page+1)):
    
    print('d_listの大きさ:', len(d_list))
    
    #変数target_urlに、アクセスさくのURLを格納する
    target_url = url.format(i)
    #print()してtarget_urlを確認する
    print(target_url)
    
    #2. 'Requests'を使って1で設定したURLにアクセスする
    r = requests.get(target_url)
    
    #requestsの間隔を1秒あける
    sleep(1)

    #3. 取得したHTMLを'BeautifulSoup'で解析する
    soup = BeautifulSoup(r.text, 'html.parser')

    #4. 全ての物件情報(20件)を取得する。
    contents = soup.find_all('div', class_='cassetteitem')
    
    #5. 各物件情報から『物件の詳細』と『各部屋の情報』
    #各物件情報をforループで取得する。
    for content in contents:
        #6. 物件情報と部屋情報を取得しておく
        #物件情報を変数detailに格納する
        #各部屋の情報を変数tableに格納する
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')
    
        #物件情報から必要な情報を取得する。
        #変数titleに物件名を格納する
        #変数addressに住所を格納する
        #変数accessにアクセス情報を格納する
        #変数ageに築年数を格納する
        title = detail.find('div', class_='cassetteitem_content-title').text
        address = detail.find('li', class_='cassetteitem_detail-col1').text
        access = detail.find('li', class_='cassetteitem_detail-col2').text
        age = detail.find('li', class_='cassetteitem_detail-col3').text
    
        #部屋情報のブロックから各部屋情報を取得する
        #変数tableから全てのtrタグを取得して、変数tr_tagsに格納
        #tr_tagsの中から最初の1つだけtr_tagに格納
        tr_tags = table.find_all('tr', class_='js-cassette_link')
        tr_tag = tr_tags[0]
    
        #各部屋情報をforループで取得する
        for tr_tag in tr_tags:
        
            #部屋情報の行から、欲しい情報を取得する
            #変数floor, price, first_fee, capacityに4つの情報を格納する
            floor, price, first_fee, capacity = tr_tag.find_all("td")[2:6]
        
            #さらに細かい情報を取得する
            #priceは賃料(fee)と管理費(management_fee)
            #first_feeは敷金(deposit)と礼金(qratuity)
            #capacityは間取り(madori)と専有面積(menseki)
            fee, manegement_fee = price.find_all('li')
            deposit, qratuity = first_fee.find_all('li')
            madori, menseki = capacity.find_all('li')
        
            #取得した全ての情報を辞書に格納する
            #変数dにこれまで取得した11項目を辞書に格納する
            d = {
            "title": title, 
            "address": address,
            "acess": access,
            "age": age,
            "floor": floor.text,
            "fee": fee.text,
            "manegement_fee": manegement_fee.text,
            "deposit": deposit.text,
            "qratuity": qratuity.text,
            "madori": madori.text,
            "menseki": menseki.text,
            }
        
            #取得した辞書をd_listに格納する
            d_list.append(d)
            
#変数d_listを使って、データフレームを作成する
df = pd.DataFrame(d_list)
df2 = df.applymap(lambda x: re.sub('\n', ' ', x))
df3 = df2.applymap(lambda x: re.sub('\r', ' ', x))
df_Final = df3.applymap(lambda x: re.sub('\t', ' ', x))
df_Final.to_csv('SUUMO_Otaku.csv', index=False, encoding='utf-8-sig')

df_Final.head()

In [ ]:
#google spread sheets への転送
#ライブラリーのインポート
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# スコープと認証情報の設定
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('your-credentials-file.json', scope)

# gspreadクライアントの初期化
client = gspread.authorize(creds)

# スプレッドシートを開く（新規作成または既存のものを開く）
spreadsheet = client.open('Step3-1①-1')

# ワークシートを取得（新規作成または既存のものを開く）
worksheet = spreadsheet.get_worksheet(0) # 0は最初のワークシートを指します


# データフレームをGoogleスプレッドシートに転送
worksheet.update([df_Final.columns.values.tolist()] + df_Final.values.tolist())